In [ ]:
%matplotlib inline

In [ ]:
import LFPy
import lfpykit
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# LFPy.Cell parameters
cellParameters = {
    'morphology': 'L5_Mainen96_LFPy.hoc',  # morphology file
    'v_init': -65,                         # initial voltage
    'cm': 1.0,                             # membrane capacitance
    'Ra': 150,                             # axial resistivity
    'passive': True,                       # insert passive channels
    'passive_parameters': {"g_pas": 1. / 3E4,
                           "e_pas": -65},  # passive params
    'dt': 2**-4,                           # simulation time res
    'nsegs_method': 'lambda_f',    # discretization rule
    'lambda_f': 1000               # frequency (Hz)
}

# create LFPy.Cell instance
cell = LFPy.Cell(**cellParameters)
cell.set_rotation(x=4.98919, y=-4.33261, z=0.)

print(f'total number of segments: {cell.totnsegs}')

# parameters for line source potential
el_params = dict(
    x = np.linspace(0, 1000, 1001),
    y = np.zeros(1001),
    z = np.zeros(1001),
    sigma = 0.3
)

cell.simulate(rec_imem=True)

# create line-source potential predictor
lsp = lfpykit.LineSourcePotential(cell, **el_params)

In [ ]:
%%prun -s cumulative -q -l 20 -T prun0
for i in range(100): 
    lsp.get_transformation_matrix()
print(open('prun0', 'r').read())

In [ ]:
M = lsp.get_transformation_matrix()
M.shape

In [ ]:
plt.matshow(M, vmin=-0.01, vmax=0.01)

In [ ]:
cell.imem.shape